# Solutions of Class Exercises - Week 3

---
---

**Author:** Dr Giordano Scarciotti (g.scarciotti@imperial.ac.uk) - Imperial College London

**Module:** ELEC70066 - Advanced Optimisation

**Version:** 1.1.2 - 31/01/2023

---
---

**Reminder:** The solutions of some these exercises may be found online. Do not look for the solutions, but ask for help to me or the GTA. Your assessment will not have a solution available online so it is of paramount importance that you learn how to solve the problems without relying on a solution being available.

The solutions of these exercises will be discussed at the end of the session. Moreover, the complete solution will be provided to you after 1 week.  **N.B.** **Do not share the solutions online**

## Exercise 1

Two investments are made, with random returns $R_1$ and $R_2$. The total return for the two investments is $R_1 + R_2$, and the probability of a loss (including breaking even, i.e., $R_1+R_2=0$) is $p^{loss} = \textbf{prob}(R_1 + R_2 \le 0)$. The goal is to find the worst-case (i.e., maximum possible) value of $p^{loss}$ consistent with the following information.

We discretize the values that $R_1$ and $R_2$ can take to $n=100$ possible values $r_1$, ..., $r_n$, uniformly spaced from $r_1 = -35$ to $r_n = +75$. $R_1$ and $R_2$ are assumed to have Gaussian [marginal distributions](https://en.wikipedia.org/wiki/Marginal_distribution) with known means $\mu_1$ and $\mu_2$ and known standard deviations $\sigma_1$ and $\sigma_2$. Hence, the discretized marginals $p^{(1)}$ and $p^{(2)}$ for $R_1$ and $R_2$ are given by

$$
\displaystyle p_i^{(k)} = \textbf{prob}(R_k = r_i) = \frac{\displaystyle e^{ \frac{-(r_i - \mu_k)^2}{2\sigma_k^2}}}{\displaystyle \sum_{j=1}^n e^{ \frac{-(r_j - \mu_k)^2}{2\sigma_k^2}}}
$$

for $k = 1,2$ and $i = 1, ..., n$. In addition it is known that $R_1$ and $R_2$ are correlated with [correlation coefficient](https://en.wikipedia.org/wiki/Correlation#Pearson's_product-moment_coefficient) $\rho$, which means that

$$
\sum_{i,j} (r_i - \mu_1) \cdot \textbf{prob}(R_1 = r_i , R_2 = r_j) \cdot (r_j - \mu_2) = \rho \sigma_1 \sigma_2
$$


Consider the data $\mu_1 =9$, $\mu_2 = 22$, $\sigma_1 = 7$, $\sigma_2 = 19$ and $\rho = -0.35$.

In [ ]:
import cvxpy as cp
import numpy as np

# Data
n = 100
rmin = -35
rmax = 75

mu1 = 9
mu2 = 22
sigma1 = 7
sigma2 = 19
rho = -0.35

**Formulate the problem as a convex optimisation problem and solve it.**
* (*Extra - IGNORE this sentence unless you finish*) You may compare your result with the case in which $R_1$ and $R_2$ are jointly Gaussian, i.e. $R_1+R_2\sim\mathcal{N}(\mu_1+\mu_2,\sigma_1^2+\sigma_2^2+2\rho\sigma_1\sigma_2)$. In that case the probabilities are uniquely identified by the Gaussian cumulative distribution, so there is no optimisation to do and the loss can be computed directly with a single `scipy.stats` function.

**N.B.** Think. This exercise is not straightforward, but you have one hour to solve it. When in doubt ask for help. Depending on how much time you have left, we will provide help to let all groups reach the solution by the end of the hour.

Below there are two hints. If you click on them, they will expland and you will be able to read them. Use them if you need.

Before starting coding try to write with pen and paper the problem. Start with the constraints and follow with the objective function. If you are unsure about what the variable of the problem is, check Hint 1.

### Hint 1: I do not know what the variable of the problem is

The variable of your problem is $P$, which is the matrix of joint probabilities defined as $P_{ij} = \textbf{prob}(R_1 = r_i, R_2 = r_j)$.

### Hint 2: Help on some constraints

The marginals $p^{(1)}$, $p^{(2)}$ are related to the joint probability matrix $P$ by summation along the two axis of the matrix.

### Solution Exercise 1

Let $P$ be the matrix of joint probabilities with $P_{ij} = \textbf{prob}(R_1 = r_i, R_2 = r_j)$. The condition that the marginals are the given ones is

$$
P \mathbf{1} = p^{(1)}, \qquad P^\top \mathbf{1} = p^{(2)}. \tag{1}
$$

The probability of a loss $R_1 + R_2 \le 0$ is given by

$$
\textbf{prob}(R_1+R_2 \le 0) = \sum_{r_i + r_j \le 0} P_{ij}
$$

In summary, the problem can be formulated as

$$
\begin{array}{ll}
\displaystyle \max & \displaystyle \sum_{r_i+r_j\le 0} P_{ij}\\
\text{s.t. } & P_{ij} \ge 0, \quad i,j=1,\dots,n\\
& P \mathbf{1} = p^{(1)}, \\
& P^\top \mathbf{1} = p^{(2)}, \\
& (r - \mu_1 \mathbf{1})^\top P (r - \mu_2 \mathbf{1}) = \rho \sigma_1 \sigma_2
\end{array}
$$

Note that we have omitted the constraint $\sum_{ij}P_{ij}=1$ because this is implied by $(1)$. **N.B.** You can omit a constraint only if you justify why it can be omitted. Without justification the omission will be considered an error.

At this point we can implement the problem in CVXPY.

In [ ]:
import cvxpy as cp
import numpy as np

In [ ]:
# Given data
n = 100
rmin = -35
rmax = 75

mu1 = 9
mu2 = 22
sigma1 = 7
sigma2 = 19
rho = -0.35

In [ ]:
# Discretized returns
r = np.linspace(rmin,rmax,n)

# Computation of the marginals p1 and p2
p1 = np.exp(-(r-mu1)*(r-mu1)/(2*sigma1*sigma1))
p1 = p1/np.sum(p1)

p2 = np.exp(-(r-mu2)*(r-mu2)/(2*sigma2*sigma2))
p2 = p2/np.sum(p2)

In [ ]:
# Construction of the objective function
P = cp.Variable((n,n))
p_loss = 0

for i in range(n):
  for j in range(n):
    if r[i] + r[j] <= 0:
      p_loss=p_loss+P[i][j]


In [ ]:
#Alternative construction of the objective function
#P = cp.Variable((n,n))
#R1,R2 = np.meshgrid(r,r)
#tot_return_mask = R1+R2 <= 0
#p_loss = cp.sum(cp.multiply(P,tot_return_mask))

In [ ]:
# Definition of the constraints
constr = [P>=0,cp.sum(P,1) == p1,cp.sum(P.T,1) == p2,(r-mu1).T@P@(r-mu2) == rho*sigma1*sigma2, cp.sum(P)==1]

# Alternative constraints
# cp.sum(P,1) == p1 can be written as P@np.ones(n)==p1
# cp.sum(P.T,1) == p2 can be written as P.T@np.ones(n)==p2

# Solution
prob = cp.Problem(cp.Maximize(p_loss),constr)
prob.solve()

0.18928791834325223

Your investments have a $18.9\%$ probability of a loss (including breaking even).

We can compare the result with the case in which $R_1$ and $R_2$ are jointly Gaussian.

In [ ]:
from scipy.stats import norm

In [ ]:
norm(mu1+mu2,np.sqrt(sigma1*sigma1 + sigma2*sigma2 + 2*rho*sigma1*sigma2)).cdf(0)

0.04080602976915407

Thus the loss in the case in which $R_1$ and $R_2$ are not jointly Gaussian is more than four times larger.

## Exercise 2

Determine whether the following sets are convex or not:

1.   A slab, i.e. a set of the form $\{x \in \mathbb{R}^n : \alpha \le a^\top x \le \beta \}$.
2.   A rectangle, i.e. a set of the form $\{x \in \mathbb{R}^n : \alpha_i \le  x_i \le \beta_i, \, i=1,\dots n \}$.
3.   A wedge, i.e. $\{x \in \mathbb{R}^n : a_1^\top x \le b_1, a_2^\top x\le b_2\}$.
4.   Consider two fixed points $x_0$ and $x_1$. Show that the set
$$
\{x: ||x-x_0||_2 \le ||x-x_1||_2 \}
$$
is a halfspace.
5.   The set of points closer to a given point than a given set, i.e.
$$
\{ x : ||x-x_0||_2 \le || x - y ||_2 \text{ for all }y\in S\}
$$
where $S \subseteq{R}^n$

6.   The set of points closer to one set than another, i.e.
$$
\{ x: \textbf{dist}(x,S) \le \textbf{dist}(x,T) \}
$$
where $S,T \subseteq \mathbb{R}^n$ and

$$
\textbf{dist}(x,S) = \inf\{||x-z||_2 : z \in S \}.
$$

7. The set $\{ x : x + S_2 \subseteq S_1 \}$, where $S_1,S_2 \subseteq \mathbb{R}^n$ with $S_1$ convex.

### Solution Exercise 2

1.   A slab is an intersection of two halfspaces, hence it is a convex set and a polyhedron.
2.   Same as above. A rectangle is a convex set and a polyhedron because it is an intersection of halfspaces.
3.   Same as above. A wedge is an intersection of two halfspaces, so it is convex set and a polyhedron.
Note that this is a cone if $b_1 = 0$ and $b_2 = 0$.
4.   Note that
$$
\begin{array}{rl}
\{x: ||x-x_0||_2 \le ||x-x_1||_2 \} &\iff (x - x_0)^\top (x - x_0) \le (x-x_1)^\top(x-x_1) \\
&\iff x^\top x - 2x_0^\top x + x_0^\top x_0 \le x^\top x - 2 x_1^\top x + x_1^\top x_1 \\
&\iff 2(x_1 - x_0)^\top x \le x_1^\top x_1 - x_0^\top x_0
\end{array}
$$
which defines a halfspace.
5.   This set is convex because it can be expressed as
$$
\bigcap_{y\in S} \{x: ||x-x_0||_2 \le ||x-y||_2 \},
$$
i.e. an intersection of sets of the type
$$
\{x: ||x-x_0||_2 \le ||x-y||_2 \}
$$
which by part 4, defines a halfspace for each fixed $y$.
6.   In general this set is not convex, as the following example in $\mathbb{R}$ shows. With $S = \{ -1, 1 \}$ and $T = \{0\}$, we have
$$
\{ x: \textbf{dist}(x,S) \le \textbf{dist}(x,T) \} = \{x \in \mathbb{R} : x \le -\frac{1}{2} \text{ or } x  \ge \frac{1}{2} \}
$$
which is not convex.
7.   This set is convex. $x + S_2 \subseteq S_1$ if $x + y \in S_1$ for all $y \in S_2$. Therefore
$$
\{ x : x + S_2 \subseteq S_1 \} =\bigcap_{y\in S_2} \{ x : x + y \in S_1 \} = \bigcap_{y\in S_2} (S_1-y)
$$
the intersection of convex sets $S_1 - y$.

## Exercise 3

Show that if $S_1$ and $S_2$ are convex sets in $\mathbb{R}^{m+n}$, then so is their partial sum

$$
S = \{(x, y_1 + y_2) : x \in \mathbb{R}^m, y_1, y_2 \in \mathbb{R}^n, (x, y_1) \in S_1, (x, y_2) \in S_2\}.
$$

### Solution Exercise 3

We consider two points $(\bar x, \bar y_1 + \bar y_2)$, $(\tilde{x}, \tilde{y}_1 + \tilde{y}_2) \in S$, i.e.

$$
(\bar x, \bar y_1) \in S_1,\quad (\bar x, \bar y_2) \in S_2, \quad (\tilde{x}, \tilde{y}_1) \in S_1,\quad (\tilde{x}, \tilde{y}_2)\in S_2.
$$

For $0\le \theta \le 1$,

$$
\theta(\bar x, \bar y_1 + \bar y_2) + (1-\theta) (\tilde{x}, \tilde{y}_1 + \tilde{y}_2) = (\theta \bar x + (1-\theta)\tilde{x}, \theta \bar y_1 + (1-\theta)\tilde{y_1} + \theta \bar y_2 + (1-\theta)\tilde{y_2})
$$
is in $S$ because, by convexity of $S_1$ and $S_2$,
$$
(\theta \bar x + (1-\theta)\tilde{x}, \theta \bar y_1 + (1-\theta)\tilde{y_1}) \in S_1, \qquad (\theta \bar x + (1-\theta)\tilde{x}, \theta \bar y_2 + (1-\theta)\tilde{y_2}) \in S_2
$$
